# Parameter exploration for TCR/CAR antagonism in the revised AKPR model
Notebook to explore the revised AKPR model TCR/CAR antagonism results as a function of parameters. We define a few plotting functions that make it easy to change values in the vector of best parameter fits and visualize the corresponding antagonism curves. 

In [ ]:
import numpy as np
import pandas as pd
import json

import sys, os
# Local modules
import sys, os
if not "../" in sys.path:
    sys.path.insert(1, "../")

from mcmc.costs_tcr_car_antagonism import antag_ratio_panel_tcr_car, cost_antagonism_tcr_car
from mcmc.mcmc_analysis import find_best_grid_point
from utils.preprocess import (michaelis_menten, string_to_tuple)
from mcmc.utilities_tcr_car_antagonism import (prepare_car_antagonism_data, load_tcr_car_molec_numbers, 
                        load_tcr_tcr_akpr_fits, check_fit_model_antagonism, plot_fit_car_antagonism)

## Data and parameter fits loading

In [ ]:
# plotting functions
def change_log_ticks(axis, base=2, which="y"):
    if which == "y" or which == "both":
        ogyticks = axis.get_yticks()
        newyticks = list(np.unique([int(x) for x in ogyticks]))
        newyticklabels = [str(base)+'$^{'+str(x)+'}$' for x in newyticks]
        axis.set_yticks(newyticks)
        axis.set_yticklabels(newyticklabels)
    if which == "x" or which == "both":
        ogxticks = axis.get_xticks()
        newxticks = list(np.unique([int(x) for x in ogxticks]))
        newxticklabels = [str(base)+'$^{'+str(x)+'}$' for x in newxticks]
        axis.set_xticks(newxticks)
        axis.set_xticklabels(newxticklabels)
    return axis
    

In [ ]:
# Extracting fixed parameters from TCR/TCR fits and calibration
# Number of TCR and CAR per T cell, CD19 per tumor, pulse KD, peptide taus
molec_counts_fi = os.path.join("..", "data", "surface_counts", "surface_molecule_summary_stats.h5")
mtc = "Geometric mean"
res = load_tcr_car_molec_numbers(molec_counts_fi, mtc, tcell_type="OT1_CAR")
tcr_number, car_number, cd19_number, l_conc_mm_params, pep_tau_map = res

# Prepare data for fitting antagonism ratios
data_file_name = os.path.join("..", "data", "antagonism", "combined_OT1-CAR_antagonism.hdf")
df = pd.read_hdf(data_file_name)
chosen_fit_conc = "1uM"
data_prep = prepare_car_antagonism_data(df, l_conc_mm_params,
                pep_tau_map, cyto="IL-2", do_plot=False,
                tcr_conc=["1nM", "1uM"], tcr_itams="10", car_itams="3")
df_data, df_err = data_prep  # Processed antagonism data

# Load TCR best fit parameters
tcr_results_file = os.path.join("..", "results", "mcmc", "mcmc_results_akpr_i.h5")
tcr_analysis_file = os.path.join("..", "results", "mcmc", "mcmc_analysis_akpr_i.json")
tcr_loads = load_tcr_tcr_akpr_fits(tcr_results_file, tcr_analysis_file)
# params: phi, kappa, cmthresh, S0p, kp, psi0, gamma_tt
# Then, [N, m, f] and I_tot
tcr_params, tcr_nmf, tcr_itot = tcr_loads

# Choose tau, and use experimentally calibrated L for CD19
cd19_tau_l = (500.0, cd19_number)

phi_car = tcr_params[0]*0.005   # phi < 100x slower for CAR than TCR, as suggested in Harris, 2018
fcmin = df_data.min()
# psi0_factor = psi0*tau / (psi0*tau + 1) = FC_min * phi_factor
# Invert: psi0 = psi0_factor / (1 - psi0_factor)
psi0_factor_car = fcmin * phi_car / (1.0 + phi_car*cd19_tau_l[0])
psi0_car = psi0_factor_car / (1.0 - psi0_factor_car*cd19_tau_l[0])

# And then smaller, so cmthresh is not pushed too high
# and there can still be antagonism at high CAR antigen densities.
psi0_car /= 3.0

# Fixed parameters for CAR: phi, kappa, psi0, gamma_cc
# Fix gamma_cc = 1.0, because s_thresh_car can compensate it.
# Faster binding, slower phosphorylation (Harris et al., 2018)
car_params = [
    tcr_params[0]*0.005,   # phi 200x slower
    tcr_params[1]*10.0,  # kappa 10x larger, so antigen's KD 100x larger
    tcr_params[6],        # gamma_tt = gamma_cc = 1.0
    psi0_car
]
# Wrapping up parameters. Fixed ones:
# phi_tcr, kappa_tcr, cmthresh_tcr, S0p, kp, psi0, gamma_tt
# then phi_car, kappa_car, psi0_car
tcr_car_params = tcr_params + car_params

# Total R (both types) and S: fixed. R, I of TCR, then R of CAR
tcr_rs = [tcr_number, tcr_itot]
tcr_car_ritots = tcr_rs + [car_number]

# N, m, f for 6Y TCR and 3-ITAM CAR: fixed ones, n_tcr, m_tcr, f_tcr, n_car
n_car = 3
tcr_car_nmf = tcr_nmf + [n_car]

# Parameter boundaries: log10 of C_m_thresh_car, I_thresh_car,
# gamma_{TC}, gamma_{CT}, tau_c_tcr, tau_c_car
fit_bounds = [(1, 100*tcr_car_ritots[2]), (1e-5, 1000.0*tcr_car_ritots[1])]
fit_bounds += [(1e-4, 1e3), (1e-2, 1e4)]
fit_bounds += [(1.0, 20.0), (5.0, 5e2)]
# Rearrange as one array of lower, one array of upper bounds
fit_bounds = [np.log10(np.asarray(a)) for a in zip(*fit_bounds)]

In [ ]:
print("TCR best parameters")
print("tcr_params:", tcr_params)
print("tcr_nmf:", tcr_nmf)
print("1 nM TCR Ag =", michaelis_menten(1e-3, *l_conc_mm_params), "ligands")
print("1 uM TCR Ag =", michaelis_menten(1e0, *l_conc_mm_params), "ligands")          

# Import analysis results
with open(os.path.join("..", "results", "mcmc", "mcmc_analysis_tcr_car_both_conc.json"), "r") as h:
    car_analysis = json.load(h)
best_kmf, best_pvec, best_cost = find_best_grid_point(car_analysis, strat="best")
kmf_tuple = string_to_tuple(best_kmf)
print("Best kmf:", best_kmf)
print("Best fit:", 10.0**best_pvec)
print("Cost:", best_cost)

In [ ]:
def plot_fit_cost_car(pvec, bounds, kmf, other_rates, rstots, nmf_fixed, cd19_tau_l, df_ratio, df_ci, mm_params, 
                     panel_fct=antag_ratio_panel_tcr_car, cost_fct=cost_antagonism_tcr_car):
    # First, compute a model panel
    other_args = [other_rates, rstots, nmf_fixed, cd19_tau_l]
    df_mod = check_fit_model_antagonism(panel_fct, pvec, kmf, 
                    df_ratio, df_ci, other_args=other_args, 
                    n_taus=101, antagonist_lvl="TCR_Antigen")
    
    # Compute the cost function too and print it
    cost_val = cost_fct(pvec, bounds, kmf, other_rates, rstots, 
                        nmf_fixed, cd19_tau_l, df_ratio, df_ci)
    
    fig, ax = plot_fit_car_antagonism(df_ratio, df_mod, mm_params, df_ci, cost=cost_val, model_ci=None)
    
    return fig, ax

## Best fit visualization

In [ ]:
# Check the best fit from the simulations first
df_model, cost_test = plot_fit_cost_car(best_pvec, fit_bounds, kmf_tuple, tcr_car_params, tcr_car_ritots, 
                                     tcr_car_nmf, cd19_tau_l, df_data, df_err, l_conc_mm_params)

## Try varying the best fit parameters

In [ ]:
# Try different parameters here
# pvec: cmthresh_car, ithresh_car, gamma_tc, gamma_ct, psi0_car, tau_c_tcr, tau_c_car
tweak_kmf_tuple = (2, 2, 2)
tweak_pvec = np.asarray(car_analysis.get(str(tweak_kmf_tuple)).get("param_estimates").get("MAP best"))
tweak_pvec[1] += 0.1  # try changing ithresh_car

df_model, cost_test = plot_fit_cost_car(tweak_pvec, fit_bounds, tweak_kmf_tuple, tcr_car_params, tcr_car_ritots, 
                                     tcr_car_nmf, cd19_tau_l, df_data, df_err, l_conc_mm_params)

In [ ]:
# Try changing the ligand number-pulse concentration conversion
mm_params_alt = [1.05e5, l_conc_mm_params[1]/10.0]
data_prep = prepare_car_antagonism_data(df, mm_params_alt,
                pep_tau_map, cyto="IL-2", do_plot=False,
                tcr_conc=["1nM", "1uM"], tcr_itams="10", car_itams="3")
df_data_alt, df_err_alt = data_prep  # Processed antagonism data
print(cd19_tau_l)
cd19_tau_l_alt = (500.0, 3e4)  # 30,000 CD19
tcr_car_ritots_alt = [3e4, 1.0, 6e4]  # [TCR number, I_tot, CAR number]
car_kmf_alt = (1, 1, 2)
pvec_alt = np.asarray(car_analysis.get(str(car_kmf_alt)).get("param_estimates").get("MAP best"))
df_model_alt, cost_test_alt = plot_fit_cost_car(pvec_alt, 
            fit_bounds, car_kmf_alt, tcr_car_params, tcr_car_ritots_alt,  
            tcr_car_nmf, cd19_tau_l_alt, df_data_alt, df_err_alt, mm_params_alt)

## Importance of $\psi_0$
Whereas $\psi_0^T$ does not matter much for TCR/TCR antagonism, it seems to be important to have $\psi_0^C > 0$, although the precise value is unimportant and can be compensated for in the fits.  

Setting it to zero in CAR and TCR just doesn't work, it makes antagonism too sensitive to ligand concentration and ligand quality because any small change in TCR completely changes the CAR output. Having a small constant $\psi_0^C$ helps antagonism modulate CAR output in part, but never too drastically. This $\psi_0^C$ is also important because it allows a large $\gamma_{TC}$, so the TCR can positively feed back on itself by deactivating the CAR-caused inhibition of the TCR. 

In [ ]:
# Using an old, hardcoded fit for illustration purposes. 
# Try just setting psi_0 = 0.1 * base value, and compensating for that with other parameters
# tcr_car_params: phi_tcr, kappa_tcr, cmthresh_tcr, S0p, kp, psi0, gamma_tt
# then phi_car, kappa_car, psi0_car
tcr_car_params_psi0 = np.asarray([1.61686906e-01, 1.0e-04, 1.92387094e+04, 1.06286931e-05, 
    1.0, 1.86356706e-07, 1.0, 8.08434532e-04, 1.0e-03, 1.0, 1.04089744e-05])
tcr_car_params_psi0[-1] *= 0.1

# pvec: cmthresh_car, ithresh_car, gamma_tc, gamma_ct, psi0_car, tau_c_tcr, tau_c_car
# Compensate
kmf_psi0 = (2, 3, 2)
pvec_psi0 = np.asarray([4.33187832, -2.89906409, -0.38345636,  3.52120797,  0.55109948,  2.37069387])
# Roughly compensating
pvec_psi0[0] += -2
pvec_psi0[1] += -3
pvec_psi0[2] += 2
pvec_psi0[3] += 0
df_model, cost_test = plot_fit_cost_car(pvec_psi0, fit_bounds, kmf_psi0, tcr_car_params_psi0, tcr_car_ritots, 
                                     tcr_car_nmf, cd19_tau_l, df_data, df_err, l_conc_mm_params)

# However, if psi0 = 0, we just couldn't compensate that. 